In [5]:
import numpy  as np
import pandas as pd
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin.gz', binary=True)

TRAIN_PATH = "data/train.csv"
TEST_PATH = "data/test.csv"


In [84]:
train = pd.read_csv(TRAIN_PATH)
eval_data = pd.read_csv(TEST_PATH)

In [91]:
import re

def text_to_wordlist(text, remove_stopwords=False, stem_words=False):
    # Clean the text, with the option to remove stopwords and to stem words.
    # Convert words to lower case and split them
    text = text.lower().split()

    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    # Return a list of words
    return(text)

In [95]:
import re
train_1 = [] 
train_2 = []
for q1 in train['question1'].values:
    train_1.append(text_to_wordlist(q1))
    

NameError: name 're' is not defined

In [93]:
train.head()


,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [17]:
train['question1'].values[1].split(" ")

['What', 'is', 'the', 'story', 'of', 'Kohinoor', '(Koh-i-Noor)', 'Diamond?']

In [55]:
x1 = []
x2 = []
for q1 in train['question1'].values.astype("str"):
    q1_count = 0.001
    q1_vec = np.zeros((300,),dtype="float64")
    for word in q1.split(" "):
        try:
             q1_vec = np.add(q1_vec,model.word_vec(word))
        except KeyError as e:
            continue
        q1_count += 1
    q1_vec = np.divide(q1_vec,q1_count)
    x1.append(q1_vec)
    
for q2 in train['question2'].values.astype("str"):
    q2_count = 0.001
    q2_vec = np.zeros((300,),dtype="float64")
    for word in q2.split(" "):
        try:
             q2_vec = np.add(q2_vec,model.word_vec(word))
        except KeyError as e:
            continue
        q2_count += 1
    q2_vec = np.divide(q2_vec,q2_count)
    x2.append(q2_vec)





In [56]:
Word2Vec = np.concatenate((x1,x2),axis=1)
y = train['is_duplicate']
print(len(Word2Vec))

404290


In [57]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
X_train = np.nan_to_num(X_train)
y_train = np.nan_to_num(y_train)
X_test = np.nan_to_num(X_test)
y_test = np.nan_to_num(y_test)
X_train, X_test, y_train, y_test = train_test_split(Word2Vec, y, test_size=0.2, random_state=0)        
clf = MLPClassifier()
clf.fit(X_train, y_train)
clf.score(X_test, y_test) 

0.77945286799079871

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?


In [60]:
x1 = []
x2 = []
for q1 in eval_data['question1'].values.astype("str"):
    q1_count = 0.001
    q1_vec = np.zeros((300,),dtype="float64")
    for word in q1.split(" "):
        try:
             q1_vec = np.add(q1_vec,model.word_vec(word))
        except KeyError as e:
            continue
        q1_count += 1
    q1_vec = np.divide(q1_vec,q1_count)
    x1.append(q1_vec)
    
for q2 in eval_data['question2'].values.astype("str"):
    q2_count = 0.001
    q2_vec = np.zeros((300,),dtype="float64")
    for word in q2.split(" "):
        try:
             q2_vec = np.add(q2_vec,model.word_vec(word))
        except KeyError as e:
            continue
        q2_count += 1
    q2_vec = np.divide(q2_vec,q2_count)
    x2.append(q2_vec)
Word2Vec_test = np.concatenate((x1,x2),axis=1)


In [61]:
eval_data_y = clf.predict_proba(Word2Vec_test)

In [83]:
SUBMISSION_PATH = "submission/"
submission = pd.DataFrame({'test_id':eval_data["test_id"], 'is_duplicate':eval_data_y[:,0]})
submission.to_csv(SUBMISSION_PATH+"baseline_submission1"+'.csv', index=False)

In [1]:
import numpy  as np
import pandas as pd
import re
import gensim

from utils import text_to_wordlist

#model = gensim.models.KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin.gz', binary=True)

TRAIN_PATH = "data/train.csv"
TEST_PATH = "data/test.csv"

train = pd.read_csv(TRAIN_PATH)
eval_data = pd.read_csv(TEST_PATH)

train_1 = [] 
train_2 = []


for q1 in train['question1'].values:
    train_1.append(text_to_wordlist(q1))
for q2 in train['question2'].values:
    train_2.append(text_to_wordlist(q1))

print(train_1[1])


what is the story of kohinoor koh - i - noor diamond 


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts_1 + texts_2 + test_texts_1 + test_texts_2)

sequences_1 = tokenizer.texts_to_sequences(texts_1)
sequences_2 = tokenizer.texts_to_sequences(texts_2)
test_sequences_1 = tokenizer.texts_to_sequences(test_texts_1)
test_sequences_2 = tokenizer.texts_to_sequences(test_texts_2)

word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))